## External application interaction with OSDU SaaS 
This notebook has been created to show how to
### 1. get a token as an external service / app to interact with service endpoints (APIs) hosted by Schlumberger  
Secrets management is a part of the DELFI control plane and this demo will show how to generate an SAuth (Schlumberger authorization) token.  More information on DELFI Authenticator service here: https://developer.delfi.slb.com/learning-center/api-guides/api-consumption/authenticationservice
### 2. call the OSDU search API 
using the OSDU schema for well logs to get the list of all well logs in the data partition
OSDU documentation here: https://community.opengroup.org/osdu/platform/system/search-service/-/blob/master/docs/api/search_openapi.yaml
### 3. call the OSDU Wellbore DMS APIs 
OSDU documentation here: https://community.opengroup.org/osdu/platform/domain-data-mgmt-services/wellbore/wellbore-domain-services/-/blob/master/docs/api/openapi.json
More documentation on Wellbore DMS including video: https://community.opengroup.org/osdu/documentation/-/tree/master/platform/api/Wellbore-DDMS/media


### Install necessary packages

pip install pandas

pip install requests

pip install io

pip install pyarrow

pip install fastparquet

In [ ]:
import asyncio
import requests
import json
import time
import base64
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from pandas import json_normalize
import io




In [ ]:
import nest_asyncio
nest_asyncio.apply()

## 1. Generate an SAuth token
An SAuth token is necessary to call any APIs that are hosted by Schlumberger.  The Authenticator service is hosted by DELFI and is based on the OAuth and OIDC protocols.  The setup the audience claim is done through the Developer Portal where all the API products are listed.  More on this here: https://developer.delfi.slb.com/learning-center/api-guides/api-consumption/api-consumption-guide

### Note: this needs to be replaced if using another Authenticator service 

In [ ]:
# read secrets from uploaded config file

from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read('osdu_saas_notebook.cfg')

secret_core_client_id = parser.get('OSDU', 'osdu_core_client')
secret_client_id = parser.get('OSDU', 'client_id')
secret_client_secret = parser.get('OSDU', 'client_secret')

# Get base url and data partition

base_url = parser.get('OSDU', 'base_url')
data_partition_id = parser.get('OSDU', 'data_partition')

In [ ]:
# code from cguipet

class SAuthAppAuthV2():
    """ Provide SAuth V2 application authentication """
    _token: str
    _token_expires: int

    def __init__(self, session: requests.Session, client_id: str, client_secret: str, token_service_url: str, scope: str = None):
        """
        :param session: requests.Session used for fetching token from SAuth token serivce
        :param client_id: client Id generated from dev portal / myapps
        :param client_secret: client secret generated from dev portal / myapps
        :param scope: If provided, must be a space-delimited list of client_ids of other clients this client wishes to access with the access_token received from this grant. The "aud" claim in the access_token will contain these client_ids for validation.
        :param token_service_url:
        """
        if client_id is None:
            raise ValueError('None is not allowed as client_id')
        if client_secret is None:
            raise ValueError('None is not allowed as client_secret')
        if token_service_url is None:
            raise ValueError('None is not allowed as token_service_url')

        self._session = session
        self._token_service_url = token_service_url
        self._client_id = client_id
        self._client_secret = client_secret
        self._scope = scope
        self._get_token()

    def header_value(self):
        return "Bearer " + self.token

    @property
    def token(self):
        if time.time() > self._token_expires:
            self._get_token()

        return self._token

    def update_headers(self, headers):
        if headers is None:
            headers = {}
        headers["Authorization"] = self.header_value()
        return headers

    def __call__(self, request):
        return self.update_headers(request.headers)

    def __str__(self):
        return "Authorization: " + self.header_value()

    def _get_token(self):
        url = self._token_service_url
        # compute authorization
        authorization = str(base64.b64encode(f"{self._client_id}:{self._client_secret}".encode("ascii")), "utf-8")
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Authorization': f'Basic {authorization}'
        }
        payload = None
        if self._scope:
            payload = dict(grant_type='client_credentials', scope=self._scope)
        else:
            payload = dict(grant_type='client_credentials')

        response = self._session.post(url, headers=headers, data=payload)
        response_dict = json.loads(response.text)

        self._token = response_dict['access_token']
        self._token_expires = time.time() + response_dict['expires_in'] - 60
        

class ServiceAuth:
    async def create_sauth_session_for_DELFI_OSDU(self, token_service_url, client_id, client_secret):
        # see full list here: https://developer.delfi.slb.com/learning-center/api-guides/api-consumption/application-client-ids
        osdu_core_client = secret_core_client_id
        de_core_client = "de-sauth-v2-scope-service-datalake.slbservice.com"
        audience = '{} {}'.format(osdu_core_client, de_core_client)

        auth = SAuthAppAuthV2(
            session=requests.Session(),
            client_id=client_id,
            client_secret=client_secret,
            token_service_url=token_service_url,
            scope=audience)

        return auth
    


async def get_service_token():
    #this is for DELFI services, https://developer.delfi.slb.com/learning-center/api-guides/api-consumption/api-consumption-guide
    sauth_token_service_url = r'https://csi.slb.com/v2/token'
    #generated from Developer Portal My Apps section, created for external web app  
    client_id = secret_client_id  # DEV team in myApps osduInnovationProject795062      
################################################################################################
# THIS SHOULD BE STORED IN A SAFE PLACE   
    client_secret = secret_client_secret
################################################################################################    
    svc_auth = ServiceAuth()
    return await svc_auth.create_sauth_session_for_DELFI_OSDU(sauth_token_service_url, client_id, client_secret)

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    token = loop.run_until_complete(get_service_token())
    print(token)
    
  

# 2. Call the OSDU search API 

## Set the data partition and base URL defined in Schlumberger OSDU SaaS
These can be replaced with other data partition IDs and base URL when deployed as for example PaaS.  Note that then section 1 above will also need to be replaced by another Authentication service that will manage the auth flow.

In [ ]:
#data_partition_id = 'neptune-innovation'
#base_url = 'https://eu5.api.enterprisedata.slb.com/api/'

# 2. Call the OSDU search API 

### append the right URLs from the base URL

In [ ]:
search_service_url = base_url + 'search/v2/query'
wellbore_dms_url = base_url + 'os-wellbore-ddms/ddms/v3/'


### define the right schemas to search for (defined by OSDU)

In [ ]:
#wellbore_kind = 'master-data--Wellbore'
#search_kind = '*:*:*:*'

#logs, pore pressure and mud logs
search_kind = 'osdu:wks:work-product-component--WellLog:*'

#trajectories
#search_kind = '*:*:work-product-component--WellboreTrajectory:*'

#polygons XYZ (drilling targets)
#search_kind = '*:*:work-product-component--PolylineSet:*'

#Wells
#search_kind= 'osdu:wks:master-data--Well:1.0.0'




### create the https header using the token generated in 1 above

In [ ]:
accessToken=token._token

headers = {
    'data-partition-id': data_partition_id,
    "authorization": f"Bearer {accessToken}",
    "Content-Type": "application/json",
}

# create the payload

search_payload = {
    'kind': search_kind,
    'limit': "200"
}



search_response = requests.post(search_service_url, headers=headers, json=search_payload)

print(search_response.status_code)

## Display search results

In [ ]:
#raw_data = json.loads(search_response.content) # raw bytes format
json_data = search_response.json() # json format


#json_data

#convert to Pandas dataframe
search_df = json_normalize(json_data, 'results')
#search_df = pd.read_json(json_data)
#search_df.info()
#print(search_df[:1])
#print('number of rows ' + str(search_df['createTime'].count()))
#print(search_df['createTime'].count())
with pd.option_context('display.max_rows', 100, 'display.max_columns', None):  # more options can be specified also
    display(search_df.transpose())

In [ ]:
wellboreIDs = search_df['data.WellboreID'].unique()

In [ ]:
# Call wellbore API and get the record
ID = 11
wellboreID = wellboreIDs[ID]

wellbore_service_url = wellbore_dms_url + 'wellbores'

wellbore_meta_url_appended = f'{wellbore_service_url}/{wellboreID}'
print(wellbore_meta_url_appended)

wellbore_meta = requests.get(wellbore_meta_url_appended, headers= headers)

print(wellbore_meta.status_code)

wellbore_json_data = wellbore_meta.json() # json format
wellbore_json_data

#convert to Pandas dataframe
wellbore_meta_df = json_normalize(wellbore_json_data)

# wellbore measurements
json_normalize(wellbore_meta_df['data.VerticalMeasurements'][0])

In [ ]:
wellbore_meta_df.transpose()

# 3. Get the well log IDs from the OSDU schema

In [ ]:
pd.set_option('display.max_colwidth', None) # turn off truncation as this is a pretty long string 

well_log_IDs = search_df[:].id

pd.DataFrame([search_df['data.Name'], search_df['id'], search_df['version']]).transpose()

### get the log values from the first item.  Note that the data is in Parquet format
using the OSDU Wellbore DMS GET .../welllogs/{record_id}/data API. 

Documentation here: https://community.opengroup.org/osdu/platform/domain-data-mgmt-services/wellbore/wellbore-domain-services/-/blob/master/docs/20210518_Wellbore_DMS.pdf  

API spec is here
https://community.opengroup.org/osdu/platform/domain-data-mgmt-services/wellbore/wellbore-domain-services/-/blob/master/docs/api/openapi.json

In [ ]:
# 2. get the log values
ID = 9
#9 PPFG
#50 Zones
well_logID = well_log_IDs[ID]

well_log_service_url =  wellbore_dms_url + 'welllogs'
well_log_url_appended = f'{well_log_service_url}/{well_logID}' + '/data?limit=1000&describe=false&orient=split'

print(well_log_url_appended)
# OSDU v3 API, note that this API returns in Parquet format
# https://parquet.apache.org/

well_log = requests.get(well_log_url_appended, headers= headers)

print(well_log.status_code)

#convert to Pandas dataframe

well_log_bytes = io.BytesIO(well_log.content)

df = pd.read_parquet(well_log_bytes)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(df)
df

In [ ]:
#df['Zones'] = df['Zones'].astype("category")

In [ ]:
# Find the metadata for the well log

log_meta = pd.json_normalize(search_df['data.Curves'][ID])
log_meta

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

x_label = log_meta['CurveUnit'][0]
y_label = log_meta['CurveUnit'][2]

ax.plot(df.iloc[:,2], -df['MD'], label='Fracture Gradient')
ax.plot(df['Pore pressure gradient'],-df['MD'], label='Pore Pressure Gradient')
ax.plot(df['Vertical stress gradient'],-df['MD'], label='Vertical stress gradient')


ax.legend(fontsize=12)
ax.set_ylabel('MD [m]', fontsize=12);
ax.set_xlabel('g/cm3', fontsize=12);
ax.set_title("Well Log", fontsize=20);

# 4. Get a well trajectory
#### 1. the record with metadata
#### 2. the bulk data

In [ ]:
trajectory_service_url = wellbore_dms_url + 'wellboretrajectories'

search_kind = '*:*:work-product-component--WellboreTrajectory:*'

# create the payload

search_payload = {
    'kind': search_kind,
    'limit': "200"
}


search_response = requests.post(search_service_url, headers=headers, json=search_payload)

print(search_response.status_code)

#raw_data = json.loads(search_response.content) # raw bytes format
json_data = search_response.json() # json format


#json_data
pd.set_option('display.max_colwidth', None) # turn off truncation as this is a pretty long string 

#convert to Pandas dataframe
search_df = json_normalize(json_data, 'results')
#search_df = pd.read_json(json_data)
#search_df.info()
#print(search_df[:1])
#print('number of rows ' + str(search_df['createTime'].count()))
#print(search_df['createTime'].count())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.DataFrame([search_df['data.Name'], search_df['modifyTime'], search_df['id']]).transpose())

#trajectory_IDs = search_df[:].id



In [ ]:
#neptune-innovation:master-data--Wellbore:50c33610-1b26-47a7-96be-f6aa1e267f26

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(search_df.iloc[0:1,:].transpose())

In [ ]:
# Select which trajectory to fetch
ID = 9


trajectoryID = trajectory_IDs[ID]
trajectory_name = search_df['data.Name'][ID]
pd.DataFrame([search_df['data.Name'], search_df['data.VerticalMeasurement.VerticalMeasurement'], search_df['id']]).transpose()

In [ ]:
# Call trajectory API and get the record

trajectory_service_url = wellbore_dms_url + 'wellboretrajectories'

trajectory_meta_url_appended = f'{trajectory_service_url}/{trajectoryID}'
print(trajectory_meta_url_appended)

trajectory_meta = requests.get(trajectory_meta_url_appended, headers= headers)

print(trajectory_meta.status_code)

trajectory_json_data = trajectory_meta.json() # json format
trajectory_json_data

#convert to Pandas dataframe
trajectory_meta_df = json_normalize(trajectory_json_data)
#trajectory_meta_df.transpose()



In [ ]:
# Find the units of measurements for the trajectory table

pd.json_normalize(trajectory_meta_df['data.AvailableTrajectoryStationProperties'][0])

In [ ]:

trajectory_url_appended = f'{trajectory_service_url}/{trajectoryID}' + '/data?limit=1000&describe=false&orient=split'
print(trajectory_url_appended)

# OSDU v3 API, note that this API returns in Parquet format
# https://parquet.apache.org/

trajectory = requests.get(trajectory_url_appended, headers= headers)

print(trajectory.status_code)

#convert to Pandas dataframe

trajectory_bytes = io.BytesIO(trajectory.content)

df = pd.read_parquet(trajectory_bytes)
df

In [ ]:
print('Trajectory TD coordinates \n X=',df.iloc[-1,9],'   Y=',df.iloc[-1,10],'    TVDSS=',df.iloc[-1,8])

In [ ]:
import numpy as np
targets_OP3 = pd.DataFrame(np.array([
    [389836, 7102757, 3316],
    [390419.7, 7102632.4, 3356],
    [390865, 7102567, 3430],
    [391576, 7102858, 3440]]), columns=['X','Y','TVDSS'])

targets_OP2 = pd.DataFrame(np.array([
    [390749, 7101685, 3371],
    [390459, 7101721, 3416],
    [389591, 7102115, 3419],]), columns=['X','Y','TVDSS'])

targets = targets_OP3

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

X = df.iloc[:,9]
Y = df.iloc[:,10]
Z = df.iloc[:,8]

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(X, Y, -Z, c=-Z, cmap='copper', label=str(trajectory_name));
ax.set_title('Well Trajectory: %s' %trajectory_name, x=0.5, y=0.9)
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_zlabel('TVDSS [m]');

ax.scatter3D(targets.iloc[:,0], targets.iloc[:,1], -targets.iloc[:,2], marker='o', s=200, label='targets');
ax.legend(loc='upper right', frameon=False)

ax.view_init(10, 45)

# 5. Get drilling targets

In [ ]:
### Get polylineSet record ID numbers

# create the search payload to get record IDs
search_kind = '*:*:work-product-component--PolylineSet:*'

search_payload = {
    'kind': search_kind,
    'limit': "200"
}

search_response = requests.post(search_service_url, headers=headers, json=search_payload)

print(search_response.status_code)

#raw_data = json.loads(search_response.content) # raw bytes format
json_data = search_response.json() # json format


#json_data
pd.set_option('display.max_colwidth', None) # turn off truncation as this is a pretty long string 
#convert to Pandas dataframe
search_df = json_normalize(json_data, 'results')

target_IDs = search_df[:].id

pd.DataFrame([search_df['data.Name'], search_df['id'], search_df['modifyTime'], search_df['data.ExtensionProperties.SlbPetrelComponent.PetrelProjectName']]).transpose()


In [ ]:
# Select a target ID

ID = 102

targetID = target_IDs[ID]
targetID_name = search_df['data.Name'][ID]

## Get record from API

storage_url = base_url + 'storage/v2/'

target_service_url = storage_url + 'records'

target_meta_url_appended = f'{target_service_url}/{targetID}'
print(target_meta_url_appended)

target_meta = requests.get(target_meta_url_appended, headers= headers)

print(target_meta.status_code)

target_json_data = target_meta.json() # json format
target_json_data

#convert to Pandas dataframe
target_meta_df = json_normalize(target_json_data)
#target_meta_df.transpose()


In [ ]:
## Get the associated FileSet


storage_url = base_url + 'file/v2/'

target_service_url = storage_url + 'files'

file_ID = target_meta_df['data.DatasetID'][0]
download_URL = 'downloadURL'

target_meta_url_appended = f'{target_service_url}/{file_ID}/{download_URL}'

print(target_meta_url_appended)

target_meta = requests.get(target_meta_url_appended, headers= headers)

print(target_meta.status_code)

target_json_data2 = target_meta.json() # json format
target_json_data2

#convert to Pandas dataframe
target_meta_df2 = json_normalize(target_json_data2)
target_meta_df2.transpose()

target_file_url = target_meta_df2.iloc[0,0]

In [ ]:
# Download target file URL and read as json

df_json = requests.get(target_file_url).json()

# normalize json file into dataframe
df = pd.json_normalize(df_json, "features")

# manipulate the coordinates part of GeoJSON into tabular format
target_list = str(df.iloc[0,2]).replace('[','')
target_list = target_list.replace(']','')

target_list = target_list.split(',')

df_target = pd.DataFrame(columns=['X','Y','Z'])
df_target['X']=target_list[0::3]
df_target['Y']=target_list[1::3]
df_target['Z']=target_list[2::3]

df_target = df_target.astype(float)

In [ ]:
# Download target as json file and store in folder

import urllib

urllib.request.urlretrieve(target_file_url, "%s" %targetID_name + ".json")

In [ ]:
# Plot target in 3d view

X = df_target['X']
Y = df_target['Y']
Z = df_target['Z']

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(X, Y, Z, c=Z, cmap='copper');
ax.set_title('Polygon: %s' %targetID_name)
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.set_zlabel('Z [m]');

# Find geometric center of target
x_c = df_target['X'].sum()/len(df_target)
y_c = df_target['Y'].sum()/len(df_target)
z_c = df_target['Z'].sum()/len(df_target)
ax.scatter3D(x_c, y_c, z_c, marker='x');
ax.text(x_c,y_c,z_c,  '%s' % (str('Centroid')), size=10, zorder=1,  
    color='k')

ax.view_init(15, -45)

#df_target

In [ ]:
## Test to search for wellbores using wellbore dms

search_service_url = 'https://eu5.api.enterprisedata.slb.com/api/os-wellbore-ddms/ddms/query/wellbores'

search_kind = '*:*:*:*'

# create the payload

search_payload = {
    'kind': search_kind,
    'limit': "200"
}

search_response = requests.post(search_service_url, headers=headers, json=search_payload)

print(search_response.status_code)

#raw_data = json.loads(search_response.content) # raw bytes format
json_data = search_response.json() # json format


#json_data
pd.set_option('display.max_colwidth', None) # turn off truncation as this is a pretty long string 

#convert to Pandas dataframe
search_df = json_normalize(json_data, 'results')
#search_df = pd.read_json(json_data)
#search_df.info()
#print(search_df[:1])
#print('number of rows ' + str(search_df['createTime'].count()))
#print(search_df['createTime'].count())
search_df

#trajectory_IDs = search_df[:].id

In [ ]:
search_df